In [1]:
import numpy as np, matplotlib.pyplot as plt
import matelms as me
import test
from scipy.sparse.linalg import gmres, bicgstab, LinearOperator

In [2]:
PI = 3.14159265359
b, eta = 1.0, 0.1
xij,yij,zij = np.array([-2.0040735703833774, 0., 0.])

In [3]:
##diagonal matrix elements
g2s = np.full(5, 3./(20*PI*eta*b))
g3t = np.full(3, 1./(2*PI*eta*b))
g3a = np.full(5, 3./(2*PI*eta*b))
g3s = np.full(7, 6./(7*PI*eta*b))

GoHH = np.diag(np.block([g2s,g3t,g3a,g3s]))

##construct two-body GHH, dimension 40x40
def GHH(xij,yij,zij):
    return np.block([[GoHH, me.GHH(xij,yij,zij, b,eta)],
                    [me.GHH(-xij,-yij,-zij, b,eta), GoHH]])

In [4]:
def GHHFH(FH):
    return np.concatenate([(test.GoHHFH(b,eta, FH[:20]) + test.GHHFH(xij,yij,zij, b,eta, FH[20:]))
                     ,(test.GHHFH(-xij,-yij,-zij, b,eta, FH[:20])+ test.GoHHFH(b,eta, FH[20:]))])

In [5]:
np.dot(GHH(xij,yij,zij),np.ones(40)).shape

(40,)

In [6]:
rhs = np.dot(GHH(xij,yij,zij),np.ones(40)) 
Ax = LinearOperator((40,40),matvec=GHHFH) ##Ax as linear operator

In [14]:
%%time
soln, exitCode = bicgstab(Ax, rhs)

CPU times: user 156 ms, sys: 459 µs, total: 157 ms
Wall time: 156 ms


In [15]:
soln

array([0.99998562, 1.00001659, 1.00003523, 1.00003394, 0.99997253,
       0.99999996, 0.99999996, 0.99999996, 1.00000049, 0.99999915,
       1.00000036, 0.9999992 , 0.99999917, 0.99998961, 0.99999984,
       0.99998617, 1.00000915, 1.00000274, 1.0000041 , 1.00000631,
       0.99985374, 1.00000476, 0.99998171, 1.00007541, 0.99997895,
       0.99999996, 0.99999996, 0.99999996, 0.99999959, 0.99999977,
       0.9999996 , 1.00000055, 1.00000099, 0.99999537, 1.00000002,
       0.99998628, 1.00000666, 1.0000026 , 1.00000396, 1.00000628])

In [16]:
x0 = rhs/np.concatenate([test.GoHHFH(b,eta, np.ones(20)), test.GoHHFH(b,eta, np.ones(20))])

In [17]:
%%time
soln, exitCode = bicgstab(Ax, rhs, x0)

CPU times: user 160 ms, sys: 679 µs, total: 160 ms
Wall time: 159 ms


In [18]:
soln

array([0.99979525, 0.9999863 , 1.00002752, 1.00010169, 0.99998629,
       1.        , 1.        , 1.        , 1.00001524, 1.00000151,
       0.99999935, 0.99999169, 0.99999841, 0.99998701, 0.99999993,
       1.0000007 , 1.00000652, 1.00000001, 1.00000005, 0.99999982,
       0.99977894, 1.00000648, 0.99996525, 1.00007708, 1.00000686,
       1.        , 1.        , 1.        , 0.9999863 , 1.00000038,
       1.0000005 , 1.00000798, 1.00000177, 1.00001256, 0.99999981,
       1.00000062, 0.9999941 , 0.9999995 , 1.00000014, 0.99999984])

In [19]:
##maybe x0 helps for many particles and time steps

In [26]:
np.allclose(test.GHHFH(xij,yij,zij, b,eta, 1/23*np.ones(20)), np.dot(me.GHH(xij,yij,zij, b,eta), 1/23*np.ones(20)))

True

In [27]:
np.allclose(test.KHHVH(xij,yij,zij, b,eta, 1/23*np.ones(20)), np.dot(me.KHH(xij,yij,zij, b,eta), 1/23*np.ones(20)))

True

In [24]:
test.KHHVH(xij,yij,zij, b,eta, np.ones(20))

array([-0.01101468,  0.00146694,  0.00146694,  0.01788082,  0.01237348,
        0.        ,  0.        ,  0.        ,  0.        ,  0.12398677,
       -0.12398677, -0.06199339,  0.06199339,  0.00160316,  0.00413752,
        0.00413752,  0.00499465,  0.00579623, -0.00310314, -0.00103438])

In [25]:
##Check whether all of them match with previous results!